# Load and test trained model

## Load libraries

In [3]:
import numpy as np
import torch

import torch.nn as nn
from enduro_lstm import *

In [4]:
def conf_cuda(use_cuda):
    
    if use_cuda:
        
        # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
        is_cuda = torch.cuda.is_available()

        # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
        if is_cuda:
            device = torch.device("cuda")
            print("GPU is available")
        else:
            device = torch.device("cpu")
            print("GPU not available, CPU used")
    else:
        device = torch.device("cpu")
        print("Selected CPU")
    return device

In [5]:
device = conf_cuda(False)

Selected CPU


In [4]:
device

device(type='cpu')

In [5]:
device.type

'cpu'

## Set configurations

In [7]:
import os
dir_path = "models/CNN_teste_m1_f1to1030_epoch5000_H200" + "/"
arr = os.listdir(f'./{dir_path}')
for i in range(len(arr)):
    print(arr[i])

CNN_teste_m1_f1to1030_epoch5000_H200
loss_file.txt


In [8]:
model_path = dir_path + "CNN_teste_m1_f1to1030_epoch5000_H200"

In [9]:
start_match = 9
end_match = 9

hidden_neurons = 200
zigzag = False
is_softmax = True

start_frame = 1
end_frame = 1030

In [10]:
data_path = r"../1-generate/data/"

use_cuda = False

In [11]:
if zigzag:
    output_size = 2
else:
    output_size = 9

## Load trained model

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
from setup_model import *
from setup_model_types import *

In [14]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model, checkpoint['optimizer']['state'][0]['step'], checkpoint['losslogger']

In [17]:
model = CNNLSTMModel(device=device, input_shape=(1,1,100,100), output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
model, last_epoch, last_logger = load_checkpoint(model, model_path)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [ ]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

start_frame = 1
end_frame = 1000

for m in range(start_match, end_match + 1):
    
    num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, m)
    frames = frames[start_frame - 1:end_frame, 30:130, 10:110]
    actions = actions[start_frame - 1:end_frame]
    
    action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
    actions = np.array(action_one_hot)
    actions = actions.reshape(len(actions), -1)
    
    frames_arr.append(frames)
    actions_arr.append(actions)
    num_of_frames_arr.append(end_frame - start_frame + 1) 

X_train = np.array(frames_arr)/255
Y_train = np.array(actions_arr)
num_of_frames_arr = np.array(num_of_frames_arr)

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [ ]:
X_train = X_train.reshape(1, end_frame-start_frame+1, -1)

In [ ]:
X_train.shape, Y_train.shape

## Prepare cell with trained model 

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
activation = nn.ReLU()
bnorm = nn.BatchNorm2d(num_features=3)
pool = nn.MaxPool2d(kernel_size=(2,2))

lstmcell = nn.LSTMCell(49*49*3, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
output = nn.Softmax()

In [ ]:
output

In [ ]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [ ]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [ ]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

## Testing outputs of model

In [ ]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [ ]:
X_train.shape

In [ ]:
hx = torch.zeros(1, hidden_neurons)
out_arr = []
for i in range(1000):
    step_input = X_train[0][i]
    step_input = step_input.view(1, 1, 100, 100)
    step_input = pool(bnorm(activation(conv(step_input))))
    step_input = step_input.reshape(1, -1)
    hx, cx = lstmcell(step_input, (hx,cx))
    out = linear(hx)
    out = output(out)
    out_arr.append(out)

In [ ]:
Y_train = Y_train.reshape(-1, len(ACTIONS_LIST))

In [ ]:
acertou = 0
errou = 0
for i in range(1000):
    if torch.argmax(Y_train[i]) == torch.argmax(out_arr[i]):
        acertou += 1
    else:
        errou += 1
        
print(acertou)
print(errou)

In [ ]:
acertou/1000

## Play Gym Enduro

In [ ]:
import gym
import time
from PIL import Image

In [ ]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [ ]:
y_min, y_max, x_min, x_max = 30, 130, 10, 110
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [ ]:
sleep_time = 0.05

In [ ]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

env.render()
time.sleep(1)

for _ in range(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx = lstmcell(frame, hx)
    out = linear(hx)
    action = output(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)